In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
page_url = 'https://search.shopping.naver.com/search/all.nhn'
params = 'query=%EC%97%90%EC%96%B4%ED%8C%9F&cat_id=&frm=NVSHATC'
params.split('&')

['query=%EC%97%90%EC%96%B4%ED%8C%9F', 'cat_id=', 'frm=NVSHATC']

In [4]:
'에어팟'.encode('utf8')

b'\xec\x97\x90\xec\x96\xb4\xed\x8c\x9f'

In [5]:
page_url = 'https://search.shopping.naver.com/search/all.nhn'
# 지워보니 cat_id와 frm도 필요없더라..

params = {
    'query' : '에어팟'
}
res = requests.get(page_url, params=params)
res


<Response [200]>

In [6]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [9]:
tag_list = soup.select('.goods_list ._itemSection')
len(tag_list)

40

In [10]:
tag=tag_list[0]

In [15]:
title_tag = tag.select_one('.tit')
goods_url = title_tag['href']
goods_name = title_tag['title']
goods_url, goods_name

('http://cr2.shopping.naver.com/adcr.nhn?x=laZEGW%2BtWgOSR6SSnbG4c%2F%2F%2F%2Fw%3D%3DsiEky1DWIRp9RPCm8ky0MMhK519%2FYQpLyuzJwjI0lwM%2FwMHFVfyRR41kkqop4BBe7b8mPFshxxZjUZilPFxJ49LOzWylEKLxqT9tnjLp1ZU9v5vPmSTHMhkGjhp5a56X4H%2Ba899jq4vPrE%2FGLlLG4%2FEZVgIO42F1ZaXskE4po%2F46fokx%2Bl4FoGy1QI9MhYsm97ezzdZ5SmNRKo6EhCioDiHib%2FRhqgYr9wnO%2FynfKSkAY3VoGsDE1XT1S6%2BOsYQTY%2BHjcviF3tCdK3HNpy49BDe8UEBRSWcEZo8%2BvK9eep8YAlHlfLvZE7oHvTvHgz2fijoOGrgwaFoZ%2B%2FQd5RATpAH0uxbhGCKlU8oD%2FYg3hwAniaaJza9DyXc4%2B890sFMHn3BTNc1ou8z07L6rEizbO%2BexhKqqqXKolPp7yRuyM%2B8sWG5xH%2BMDbHiNNiHNZeSlzRCmbo9tv%2Flwtg6f%2BJDJpivGNqpKCAAb85PIlxyhmhd5rPVzI8i4BESS239iSmFOWt5SQ7h9NlJklPiIKyYWpqZg7V7k688qcc3DL6Tr6EA5flfkORtuLbnKw6P2b3CvjwS2WtbSXzYjLNwsTUnaO6iSM7mfoYBKC6ortqtTwHymWQUZa3Xmw4kclqHoMSwVEf6V6dWNLrR48RgpR6lUzeAz8in5VLtp6p%2F8exCeAYHhOAF5ksHaPhHo2JPalOVV7Ax3vTX3vA2N5iwtHqwkD6A%3D%3D',
 '애플 에어팟 1세대 (MMEF2KH/A)')

In [16]:
# scr 확인해보면 '이미지 준비중' 으로 나온다. 자바스크립트로 오리지널에 있는 주소로 변경하는 것 같다. src로 하면 안된다!!
tag.select_one('.img img')

<img alt="애플 에어팟 1세대 (MMEF2KH/A)" class="_productLazyImg" data-original="https://shopping-phinf.pstatic.net/main_1077690/10776906666.20161214174829.jpg?type=f140" height="140" src="https://ssl.pstatic.net/imgshopping/static/pc2014/img/search/p_120_v1.png" width="140"/>

In [22]:
goods_img_url = tag.select_one('.img img')['data-original']
goods_img_url

'https://shopping-phinf.pstatic.net/main_1077690/10776906666.20161214174829.jpg?type=f140'

In [23]:
goods_price = int(tag.select_one('.price .num').text.replace(',',''))
goods_price

214310

In [27]:
#tag.select('.depth a')
category_list = [t.text for t in tag.select('.depth a')] # 리스트 컴프리헨션
category_list

['디지털/가전', '음향가전', '블루투스셋', '블루투스이어폰/이어셋']

In [65]:
spec_list = []

for spec_tag in tag.select('.detail a'):
    spec = spec_tag.text
    try : 
        key, value = spec.split(':', 1)
        spec_list.append([key.strip(),value.strip()])    
    except ValueError:
        key, value = spec_list[-1]
        if not isinstance(value, list):
            value = [value.strip()]
        value.append(spec.strip())
        spec_list[-1] = [key, value]
dict(spec_list)

{'용도': '통화+음악(스테레오)',
 '채널': '스테레오',
 '무선방식': '블루투스(애플W1칩탑재)',
 '유닛형태': '코드프리',
 '구성품': '충전케이스포함',
 '색상': '화이트계열',
 '품목': '무선이어폰',
 '음질개선': 'AAC',
 '부가기능': ['Siri호출(두번터치)',
  '빔포밍마이크',
  '가속도계센서(동작,음성)',
  '광학센서',
  '급속충전(15분충전시3시간재생)',
  '터치버튼',
  '자동전원']}

In [66]:
import re
graph_tag = tag.select_one('.graph')
star_tag = graph_tag.select_one('.star_graph span')
star_tag['style']

'width:96.0%'

In [70]:
matched = re.match(r'width:([\d\.]+)%', star_tag['style'])
if matched:
    star_score = float(matched.group(1)) * 5/100 # 74.0 : 100 = x : 5else:
else:
    star_score = None
star_score

4.8

In [71]:
review_count = int(graph_tag.select_one('em').text.replace(',',''))
review_count

11932

In [76]:
created_at = tag.select_one('.etc .date').text.replace('등록일', '').strip()
created_at

'2016.12.'

In [78]:
tag.select_one('.jjim')
# 찜하기 확인하기 위해서 네트워크 탭의 XHR을 열어서 keep.nhn을 확인

<a class="jjim _jjim" data-nv-mid="10776906666" href="#"><span></span>찜하기<em class="_keepCount" nvmid="10776906666"></em></a>

In [80]:
jjim_tag = tag.select_one('.jjim')
nv_mid = jjim_tag['data-nv-mid']

In [82]:
is_ad = 'ad' in tag['class']
is_ad

False

## 모아서 함수로

In [89]:
goods_list=[]
for tag in tag_list:
    is_ad = 'ad' in tag['class']
    title_tag = tag.select_one('.tit')
    goods_url = title_tag['href']
    
    goods_name = title_tag.text.strip()
    
    goods_img_url = tag.select_one('.img img')['data-original']
    goods_price = int(tag.select_one('.price .num').text.replace(',',''))
    category_list = [t.text for t in tag.select('.depth a')]

    spec_list = []
    for spec_tag in tag.select('.detail a'):
        spec = spec_tag.text
        try : 
            key, value = spec.split(':', 1)
            spec_list.append([key.strip(),value.strip()])    
        except ValueError:
            key, value = spec_list[-1]
            if not isinstance(value, list):
                value = [value.strip()]
            value.append(spec.strip())
            spec_list[-1] = [key, value]
            
    spec_dict = dict(spec_list)

    graph_tag = tag.select_one('.graph') # NoneType' object is not subscriptable로 아래를 if로 묶어줌.
    if graph_tag :
        star_tag = graph_tag.select_one('.star_graph span')
        if star_tag:
            matched = re.match(r'width:([\d\.]+)%', star_tag['style'])
            if matched:
                star_score = float(matched.group(1)) * 5/100 
            else : 
                star_score = None
        else:
            star_score = None
            
        review_count = int(graph_tag.select_one('em').text.replace(',',''))
    else :
        start_score = None
        review_count = None
    
    try : 
        created_at = tag.select_one('.etc .date').text.replace('등록일', '').strip()
    except AttributeError:
        created_at = None

    jjim_tag = tag.select_one('.jjim')
    nv_mid = jjim_tag['data-nv-mid']
    
    goods_list.append({
        'is_ad' : is_ad,
        'url' : goods_url,
        'name' : goods_name,
        'img_url' : goods_img_url,
        'price' : goods_price,
        'category_list' : category_list,
        'spec_dict' : spec_dict,
        'star+score' : star_score,
        'review_count' : review_count,
        'create_at' : created_at,
        'nv_mid' : nv_mid        
    })
goods_list[0]

{'is_ad': False,
 'url': 'http://cr2.shopping.naver.com/adcr.nhn?x=laZEGW%2BtWgOSR6SSnbG4c%2F%2F%2F%2Fw%3D%3DsiEky1DWIRp9RPCm8ky0MMhK519%2FYQpLyuzJwjI0lwM%2FwMHFVfyRR41kkqop4BBe7b8mPFshxxZjUZilPFxJ49LOzWylEKLxqT9tnjLp1ZU9v5vPmSTHMhkGjhp5a56X4H%2Ba899jq4vPrE%2FGLlLG4%2FEZVgIO42F1ZaXskE4po%2F46fokx%2Bl4FoGy1QI9MhYsm97ezzdZ5SmNRKo6EhCioDiHib%2FRhqgYr9wnO%2FynfKSkAY3VoGsDE1XT1S6%2BOsYQTY%2BHjcviF3tCdK3HNpy49BDe8UEBRSWcEZo8%2BvK9eep8YAlHlfLvZE7oHvTvHgz2fijoOGrgwaFoZ%2B%2FQd5RATpAH0uxbhGCKlU8oD%2FYg3hwAniaaJza9DyXc4%2B890sFMHn3BTNc1ou8z07L6rEizbO%2BexhKqqqXKolPp7yRuyM%2B8sWG5xH%2BMDbHiNNiHNZeSlzRCmbo9tv%2Flwtg6f%2BJDJpivGNqpKCAAb85PIlxyhmhd5rPVzI8i4BESS239iSmFOWt5SQ7h9NlJklPiIKyYWpqZg7V7k688qcc3DL6Tr6EA5flfkORtuLbnKw6P2b3CvjwS2WtbSXzYjLNwsTUnaO6iSM7mfoYBKC6ortqtTwHymWQUZa3Xmw4kclqHoMSwVEf6V6dWNLrR48RgpR6lUzeAz8in5VLtp6p%2F8exCeAYHhOAF5ksHaPhHo2JPalOVV7Ax3vTX3vA2N5iwtHqwkD6A%3D%3D',
 'name': '애플 에어팟 1세대 (MMEF2KH/A)',
 'img_url': 'https://shopping-phinf.pstatic.net/main_1077690/10776906666.201

In [90]:
import pandas as pd

In [94]:
df= pd.DataFrame(goods_list)
print(df.shape)
df.head(3)

(40, 11)


,category_list,create_at,img_url,is_ad,name,nv_mid,price,review_count,spec_dict,star+score,url
0,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]",2016.12.,https://shopping-phinf.pstatic.net/main_107769...,False,애플 에어팟 1세대 (MMEF2KH/A),10776906666,214310,11932.0,"{'용도': '통화+음악(스테레오)', '채널': '스테레오', '무선방식': '블...",4.8,http://cr2.shopping.naver.com/adcr.nhn?x=laZEG...
1,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]",2019.04.,https://shopping-phinf.pstatic.net/main_186220...,False,애플 에어팟 2세대 유선충전 (MV7N2KH/A),18622086330,209000,151.0,"{'종류': '이어셋', '용도': '통화+음악(스테레오)', '유닛형태': '코드...",4.8,http://cr2.shopping.naver.com/adcr.nhn?x=m8WjU...
2,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]",2019.04.,https://shopping-phinf.pstatic.net/main_184837...,False,해외애플 에어팟 2세대 무선충전,18483728924,235320,108.0,"{'종류': '이어셋', '용도': '통화+음악감상겸용', '무선방식': '블루투스'}",4.9,http://cr2.shopping.naver.com/adcr.nhn?x=xGh%2...


In [96]:
','.join(df['nv_mid'])

'10776906666,18622086330,18483728924,18754856889,81919971590,18428627338,18344345265,81946053258,17060334679,81958588252,12654509207,81786680577,17011209371,81934833520,13838748967,81765026705,18735689119,81919971590,80628902191,81996763612,18740571933,81988008453,18367305169,16341587630,82005912375,18899954783,16950823115,81736134795,16611371198,18607171841,16963250393,18146893827,18314856361,18711537902,18802429798,18845425918,15121518343,18679867694,14176537731,18429947106'

In [101]:
keep_url = 'https://search.shopping.naver.com/search/ajax/keep.nhn'
data = {
    'nvMid' : ','.join(df['nv_mid']),
    'serviceCode' : 'SHOPPING',
}
res = requests.post(keep_url, data=data) #get 이 아니라 post 이다
res

<Response [200]>

In [104]:
keep_count_list = res.json()['keepCount']
len(keep_count_list)

40

In [105]:
df['keep_count'] = keep_count_list

In [107]:
names = ['name','nv_mid', 'keep_count']
df[names]

,name,nv_mid,keep_count
0,애플 에어팟 1세대 (MMEF2KH/A),10776906666,3097
1,애플 에어팟 2세대 유선충전 (MV7N2KH/A),18622086330,247
2,해외애플 에어팟 2세대 무선충전,18483728924,165
3,애플 에어팟 2세대 무선충전 (MRXJ2KH/A),18754856889,35
4,캔디팟 가성비 에어로팟 에어팟 i10 i12 i9 tws 차이팟x,81919971590,268
5,해외애플 에어팟 MMEF2KH,18428627338,237
6,해외애플 에어팟 2세대 유선충전 케이스,18344345265,384
7,애플 에어팟 MMEF2KH/A 왼쪽 오른쪽 단품 한쪽 국내정식발매품 AS 가능,81946053258,22
8,마이룸 무선 블루투스 이어폰 차이팟 애플 아이폰 에어팟 x 에어로팟 에이블팟,17060334679,487
9,"[입고시발송예약/유선,무선충전버전/순서발송] 애플 국내정품 에어팟 2세대 Apple...",81958588252,291


In [110]:
df.drop(['img_url', 'url'], axis = 'columns', inplace=True)# 긴 url 때문에 오류 나옴.
df.to_excel('네이버쇼핑검색결과-에어팟.xlsx')